In [ ]:
su hduser

    start-dfs.sh
    start-yarn.sh

    jps 

    ls 


hdfs dfs -ls / 

    hdfs dfs -rm -r /input

    hdfs dfs -rm -r /output
    
    hdfs dfs -mkdir -p /input
    
    hdfs dfs -ls / 
    
    nano weather.txt
cat weather.txt
    
    hdfs dfs -put weather.txt /input/
    
    hdfs dfs -ls /input/
    
    nano mapper.py

    nano reducer.py

    chmod +x  mapper.py

    chmod +x reducer.py


    
    whereis hadoop
    
    ls  /usr/local/hadoop/

    ls  /usr/local/hadoop/share/ 

    ls  /usr/local/hadoop/share/hadoop/
    
    ls  /usr/local/hadoop/share/hadoop/tools/

    ls  /usr/local/hadoop/share/hadoop/tools/lib/ 

    ls  /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.4.jar 
    
    hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.4.jar -input /input/weather.txt -output /output/weather_output -mapper mapper.py -reducer reducer.py -file mapper.py -file reducer.py 

    hdfs dfs -ls /
    
    hdfs dfs -ls /output/weather_output/
    
    hdfs dfs -cat /output/weather_output/part-00000
    
    stop-dfs.sh
    stop-yarn.sh
    
*******************************Text***************************************

Year,Month,Day,Max Temp (°C),Min Temp (°C),Rainfall (mm)
1950,01,01,25,-18,43
1950,01,02,26,-17,44
1950,01,03,27,-12,32
1950,01,04,28,-20,41
1950,01,05,29,-13,40
1950,01,06,30,-16,45
1950,01,07,31,-14,33
1950,01,08,32,-19,38
1950,01,09,33,-20,28
1950,01,10,34,-19,40

******************************mapper***************************************

#!/usr/bin/env python3
import sys

# Skip the header
for idx, line in enumerate(sys.stdin):
    if idx == 0:
        continue  # Skip header
    parts = line.strip().split(",")
    if len(parts) != 6:
        continue  # Skip malformed lines

    year = parts[0]
    try:
        max_temp = float(parts[3])
        min_temp = float(parts[4])
    except ValueError:
        continue  # Skip lines with non-numeric temperature

    # Emit key-value pairs
    print(f"{year}\t{max_temp},{min_temp},1")
    
********************************reducer***********************************

#!/usr/bin/env python3
import sys

from collections import defaultdict

temp_data = defaultdict(lambda: [0, 0, 0])  # max_sum, min_sum, count

for line in sys.stdin:
    line = line.strip()
    if not line:
        continue
    parts = line.split("\t")
    if len(parts) != 2:
        continue
    year, values = parts
    try:
        max_temp, min_temp, count = map(float, values.split(","))
        temp_data[year][0] += max_temp
        temp_data[year][1] += min_temp
        temp_data[year][2] += count
    except ValueError:
        continue

# Output: Year -> avg max, avg min
for year in sorted(temp_data):
    max_sum, min_sum, count = temp_data[year]
    avg_max = max_sum / count
    avg_min = min_sum / count
    print(f"{year}\tAvg Max Temp: {avg_max:.2f}, Avg Min Temp: {avg_min:.2f}")

In [ ]:


**AIM:**
To design and develop a distributed application to identify the coolest and hottest year from a given weather dataset using the MapReduce programming model.

---

**PROBLEM STATEMENT:**
Design and develop a distributed application to find the coolest and hottest year from the available weather data. Use weather data from the internet and process it using MapReduce.

---

**THEORY:**

**1. MapReduce:**
MapReduce is a distributed computing model designed to process large datasets in parallel by dividing the work across multiple nodes in a computing cluster. It consists of two core functions:

* **Map:** Processes input data and produces intermediate key-value pairs.
* **Reduce:** Aggregates the intermediate data to generate the final output.

**Key Features:**

* Parallel processing across a cluster
* Fault tolerance
* Scalability for large datasets

**2. Hadoop Streaming:**
Hadoop Streaming is a utility that allows developers to create and run MapReduce jobs with any executable or script that uses standard input/output. This enables programming in languages like Python, Perl, Bash, or Ruby rather than Java. For this practical, Python is used due to its simplicity and clarity.

---

**WORKFLOW: Finding the Coolest and Hottest Year Using MapReduce**

**Input File (HDFS):**

* The weather dataset is uploaded to the Hadoop Distributed File System (HDFS).
* Hadoop splits the file into blocks and distributes them across the cluster.

**Step 1: Mapper Phase**

* Reads each line from the input dataset.
* Extracts relevant fields: Year, Max Temperature, Min Temperature.
* Emits key-value pairs in the form:
  `Year → (max_temp, min_temp, count=1)`

**Step 2: Shuffle and Sort** *(Handled by Hadoop)*

* Groups data by key (year).
* Sends all values for the same key to a single reducer.

**Step 3: Reducer Phase**

* Aggregates all temperature values for each year.
* Calculates average, maximum, or minimum values as required.
* Emits:
  `Year → (average_max_temp, average_min_temp)`

**Step 4: Final Output**

* The output is stored in HDFS.
* You can determine the hottest and coolest year by checking the year with the highest average max temperature and the lowest average min temperature respectively.



In [ ]:
Here’s a detailed explanation of the theory and **expected viva questions** with **their answers**, based on your problem statement and the dataset you've provided:

---

### 🔬 **THEORY (Detailed for Viva)**

**MapReduce Framework:**
MapReduce is a framework by Google (implemented in Hadoop) used for processing massive amounts of data in a distributed environment. It breaks a job into two phases:

* **Map**: Each Mapper processes a portion of the input data and emits intermediate key-value pairs.
* **Reduce**: These intermediate pairs are then grouped by key, and each group is processed by a Reducer to generate final output.

**Hadoop:**
Hadoop is an open-source framework that uses HDFS (Hadoop Distributed File System) for storing large datasets and MapReduce for processing them.

**Hadoop Streaming:**
Hadoop Streaming allows using any programming language (like Python) to write Mapper and Reducer scripts. As long as the script reads from `stdin` and writes to `stdout`, it can be used with Hadoop.

**Weather Dataset Processing Use-Case:**
In this application, the dataset contains weather records with **date**, **maximum temperature**, and **minimum temperature**.
Goal: Identify the **coolest year** and the **hottest year** based on temperature trends.

**Map Phase:**

* Each line is parsed.
* Extract year, max temp, and min temp.
* Emit key-value pair like:
  `1950 → 34,-20,1`

**Reduce Phase:**

* For each year:

  * Total all max temps and min temps.
  * Count entries.
  * Compute average max and min temps.
* Final result for each year:
  `1950 → Avg Max Temp, Avg Min Temp`

From the final list, pick:

* Year with **highest avg max temp → Hottest year**
* Year with **lowest avg min temp → Coolest year**

---

### 📘 **Example from Your Dataset:**

```
Sample Line: 1950,01,10,34,-19,40  
Extracted: Year = 1950, Max = 34, Min = -19  
Emitted by Mapper: 1950 → 34,-19,1
```

Reducer combines all such values for 1950 and computes the average.

---

### 🎤 **VIVA QUESTIONS & ANSWERS**

---

**Q1. What is MapReduce and why is it used?**
**A:** MapReduce is a programming model used for processing large-scale data in parallel. It simplifies distributed processing by splitting the job into two main tasks—Map and Reduce. It's used because it efficiently handles huge datasets and supports scalability and fault-tolerance.

---

**Q2. What is the role of the Mapper and the Reducer?**
**A:**

* **Mapper** reads input line-by-line, processes it, and emits key-value pairs.
* **Reducer** receives grouped data by key and performs aggregation or computation.

---

**Q3. Why are we using Hadoop Streaming and not Java?**
**A:** Hadoop Streaming allows the use of any language (like Python), not just Java. This is helpful for quick development and simplicity, especially for those who are more comfortable in scripting languages.

---

**Q4. What key-value pair does the Mapper emit in your code?**
**A:** For each valid line, the Mapper emits:
`year → max_temp, min_temp, count`
E.g., `1950 → 34,-19,1`

---

**Q5. How does the Reducer process the values?**
**A:**

* Aggregates all values per year.
* Sums max temps, min temps, and counts.
* Calculates average temperatures using:
  `avg_max = total_max / count`
  `avg_min = total_min / count`

---

**Q6. How do you determine the coolest and hottest years?**
**A:**

* After Reducer output, parse all yearly results.
* The year with **highest avg max temp** is the **hottest**.
* The year with **lowest avg min temp** is the **coolest**.

---

**Q7. What challenges do you face while processing large datasets?**
**A:**

* Handling data distribution across nodes.
* Fault tolerance and node failures.
* Ensuring load balancing.
* Memory and disk space limitations.

---

**Q8. How is fault tolerance achieved in Hadoop?**
**A:** Hadoop replicates data blocks across multiple nodes. If one node fails, another node's replica is used to continue the job without failure.

---

**Q9. What format should the dataset be in for MapReduce?**
**A:** Plain text with structured data, usually CSV or tab-separated. Each line should be independently processable.

---

**Q10. What will happen if the data has malformed lines or missing values?**
**A:** Such lines are either skipped in the Mapper or handled with exception handling (try-except in Python), ensuring they don’t break the processing flow.



In [ ]:
Here's a **line-by-line explanation with comments** for both the **Mapper** and **Reducer** code used in your MapReduce application to find the hottest and coolest years:

---

### 🟨 **Mapper Script: `mapper.py`**

```python
#!/usr/bin/env python3
```

💬 This line specifies the script should be run using Python 3 — it helps in UNIX/Linux environments when running the script as an executable.

```python
import sys
```

💬 Imports the `sys` module to read input line-by-line from standard input (`stdin`) — essential for Hadoop Streaming.

```python
# Skip the header
for idx, line in enumerate(sys.stdin):
```

💬 Loops through each line from the input file. `enumerate()` provides both line index (`idx`) and the content.

```python
    if idx == 0:
        continue  # Skip header
```

💬 Skips the first line, assuming it's the header of the CSV (e.g., "Year,Month,Day,Max Temp,Min Temp,Rainfall").

```python
    parts = line.strip().split(",")
```

💬 Removes any leading/trailing whitespace and splits the line by commas into a list.

```python
    if len(parts) != 6:
        continue  # Skip malformed lines
```

💬 Ensures the line has exactly 6 values; if not, it's considered corrupt and skipped.

```python
    year = parts[0]
```

💬 Extracts the **year** from the first column (index 0).

```python
    try:
        max_temp = float(parts[3])
        min_temp = float(parts[4])
```

💬 Converts **Max Temp** and **Min Temp** (columns 3 and 4) to floating-point numbers.

```python
    except ValueError:
        continue  # Skip lines with non-numeric temperature
```

💬 If conversion fails (e.g., missing or non-numeric data), that line is ignored.

```python
    # Emit key-value pairs
    print(f"{year}\t{max_temp},{min_temp},1")
```

💬 Emits the output in `key\tvalue` format:

* **Key** = Year
* **Value** = max\_temp, min\_temp, count (1 occurrence for averaging later)

---

### 🟩 **Reducer Script: `reducer.py`**

```python
#!/usr/bin/env python3
```

💬 Indicates to use Python 3 to run the script.

```python
import sys
```

💬 Allows reading input from Hadoop Streaming’s `stdin`.

```python
from collections import defaultdict
```

💬 Imports `defaultdict`, which simplifies initializing a dictionary with default values.

```python
temp_data = defaultdict(lambda: [0, 0, 0])  # max_sum, min_sum, count
```

💬 Creates a dictionary where:

* `temp_data[year][0]`: Sum of all max temps
* `temp_data[year][1]`: Sum of all min temps
* `temp_data[year][2]`: Count of entries for that year

```python
for line in sys.stdin:
```

💬 Iterates through each line from the mapper's output.

```python
    line = line.strip()
    if not line:
        continue
```

💬 Trims whitespace; skips empty lines.

```python
    parts = line.split("\t")
    if len(parts) != 2:
        continue
```

💬 Splits by tab character (`\t`) to separate the key and value. Skips lines that don't have exactly two parts.

```python
    year, values = parts
```

💬 Extracts the year and the values (e.g., "34,-20,1").

```python
    try:
        max_temp, min_temp, count = map(float, values.split(","))
```

💬 Converts the comma-separated string into three float values.

```python
        temp_data[year][0] += max_temp
        temp_data[year][1] += min_temp
        temp_data[year][2] += count
```

💬 Updates the totals for each year.

```python
    except ValueError:
        continue
```

💬 If parsing fails due to bad data, skips that line.

---

### 🔽 **Output Phase**

```python
for year in sorted(temp_data):
```

💬 Loops through the years in sorted order.

```python
    max_sum, min_sum, count = temp_data[year]
```

💬 Retrieves the accumulated values for that year.

```python
    avg_max = max_sum / count
    avg_min = min_sum / count
```

💬 Computes the **average max** and **average min** temperatures.

```python
    print(f"{year}\tAvg Max Temp: {avg_max:.2f}, Avg Min Temp: {avg_min:.2f}")
```

💬 Outputs final results in a clean format with two decimal places.



In [ ]:
CODE: 
mapper.py 
reducer.py 
1. Open Terminal and switch to Hadoop user 
pvg@pvg-HP-ProDesk-400-G4-SFF:~$ su hduser 
Password:  
hduser@pvg-HP-ProDesk-400-G4-SFF:/home/pvg$ cd 
2. Start HDFS 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ start-dfs.sh 
Starting namenodes on [localhost] 
Starting datanodes 
Starting secondary namenodes [pvg-HP-ProDesk-400-G4-SFF] 
2025-04-23 15:17:37,545 WARN util.NativeCodeLoader: Unable to load native-hadoop 
library for your platform... using builtin-java classes where applicable 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ start-yarn.sh 
Starting resourcemanager 
Starting nodemanagers 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ jps 
16580 NodeManager 
16932 Jps 
15815 NameNode 
16216 SecondaryNameNode 
16425 ResourceManager 
15998 DataNode  
3. Create an input directory 
#Delete any previous input or output files if present using: hdfs dfs -rm -r /input /output 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ hdfs dfs -mkdir -p /input 
2025-04-23 15:20:45,988 WARN util.NativeCodeLoader: Unable to load native-hadoop 
library for your platform... using builtin-java classes where applicable 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ hdfs dfs -ls / 
2025-04-23 15:21:00,439 WARN util.NativeCodeLoader: Unable to load native-hadoop 
library for your platform... using builtin-java classes where applicable 
Found 1 items 
drwxr-xr-x   - hduser supergroup          
0 2025-04-23 15:20 /input 
4. Create a text file, paste the weather data and upload it to HDFS 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ nano weather_data.txt 
#weather_data.txt file will open in nano text editor 
#Copy the text from the provided weather data text file and paste it here 
#Press Ctrl + X 
#Press Y 
#Press Enter key 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ hdfs dfs -put weather_data.txt /input/ 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ hdfs dfs -ls /input/ 
2025-04-23 15:21:32,321 WARN util.NativeCodeLoader: Unable to load native-hadoop 
library for your platform... using builtin-java classes where applicable 
Found 1 items -rw-r--r--   1 hduser supergroup       
/input/weather_data.txt 
1685 2025-04-23 15:21 
5. Similary, create a mapper.py and reducer.py file 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ nano mapper.py 
#This will open the mapper.py file in nano editor; write the Mapper python 
code here 
#Here’s the code for mapper.py 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ nano reducer.py 
#Write the reducer python code 
#Now, make the python script executable 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ chmod +x mapper.py 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ chmod +x reducer.py 
6. Run Hadoop streaming jar using the mapper and reducer scripts 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ whereis hadoop 
hadoop: /usr/local/hadoop /usr/local/hadoop/bin/hadoop.cmd 
/usr/local/hadoop/bin/Hadoop 
# - is used to specify flags 
# \ at the end of a line is used to split a long command into multiple lines 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ hadoop jar 
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.4.jar \ 
> -input /input/weather_data.txt \ 
> -output /output/weather_output \ 
> -mapper mapper.py \ 
> -reducer reducer.py \ 
> -file mapper.py \ 
> -file reducer.py 
2025-04-23 15:24:51,122 WARN streaming.StreamJob: -file option is deprecated, 
please use generic option -files instead. 
2025-04-23 15:24:51,217 WARN util.NativeCodeLoader: Unable to load native-hadoop 
library for your platform... using builtin-java classes where applicable 
packageJobJar: [mapper.py, reducer.py] [] /tmp/streamjob9268891282553717103.jar 
tmpDir=null 
2025-04-23 15:24:51,701 INFO impl.MetricsConfig: Loaded properties from hadoop
metrics2.properties 
2025-04-23 15:24:51,769 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot 
period at 10 second(s). 
2025-04-23 15:24:51,769 INFO impl.MetricsSystemImpl: JobTracker metrics system 
started 
2025-04-23 15:24:51,784 WARN impl.MetricsSystemImpl: JobTracker metrics system 
already initialized! 
2025-04-23 15:24:51,993 INFO mapred.FileInputFormat: Total input files to 
process : 1 
2025-04-23 15:24:52,053 INFO mapreduce.JobSubmitter: number of splits:1 
2025-04-23 15:24:52,176 INFO mapreduce.JobSubmitter: Submitting tokens for job: 
job_local1388892213_0001 
2025-04-23 15:24:52,176 INFO mapreduce.JobSubmitter: Executing with tokens: [] 
2025-04-23 15:24:52,326 INFO mapred.LocalDistributedCacheManager: Localized 
file:/home/hduser/mapper.py as 
file:/app/hadoop/tmp/mapred/local/job_local1388892213_0001_3cc73aa2-b9eb-4451
bd53-20464abdb17f/mapper.py 
2025-04-23 15:24:52,360 INFO mapred.LocalDistributedCacheManager: Localized 
file:/home/hduser/reducer.py as 
file:/app/hadoop/tmp/mapred/local/job_local1388892213_0001_1494c792-fedb-4543
be63-f5178e81a6a0/reducer.py 
2025-04-23 15:24:52,404 INFO mapreduce.Job: The url to track the job: 
http://localhost:8080/ 
2025-04-23 15:24:52,405 INFO mapred.LocalJobRunner: OutputCommitter set in config 
null 
2025-04-23 15:24:52,408 INFO mapreduce.Job: Running job: job_local1388892213_0001 
2025-04-23 15:24:52,410 INFO mapred.LocalJobRunner: OutputCommitter is 
org.apache.hadoop.mapred.FileOutputCommitter 
2025-04-23 15:24:52,414 INFO output.FileOutputCommitter: File Output Committer 
Algorithm version is 2 
2025-04-23 15:24:52,414 INFO output.FileOutputCommitter: FileOutputCommitter skip 
cleanup _temporary folders under output directory:false, ignore cleanup failures: 
false 
2025-04-23 15:24:52,450 INFO mapred.LocalJobRunner: Waiting for map tasks 
2025-04-23 15:24:52,452 INFO mapred.LocalJobRunner: Starting task: 
attempt_local1388892213_0001_m_000000_0 
2025-04-23 15:24:52,476 INFO output.FileOutputCommitter: File Output Committer 
Algorithm version is 2 
2025-04-23 15:24:52,476 INFO output.FileOutputCommitter: FileOutputCommitter skip 
cleanup _temporary folders under output directory:false, ignore cleanup failures: 
false 
2025-04-23 15:24:52,484 INFO mapred.Task:  Using ResourceCalculatorProcessTree : 
[ ] 
2025-04-23 15:24:52,488 INFO mapred.MapTask: Processing split: 
hdfs://localhost:54310/input/weather_data.txt:0+1685 
2025-04-23 15:24:52,503 INFO mapred.MapTask: numReduceTasks: 1 
2025-04-23 15:24:52,536 INFO mapred.MapTask: (EQUATOR) 0 kvi 26214396(104857584) 
2025-04-23 15:24:52,536 INFO mapred.MapTask: mapreduce.task.io.sort.mb: 100 
2025-04-23 15:24:52,536 INFO mapred.MapTask: soft limit at 83886080 
2025-04-23 15:24:52,536 INFO mapred.MapTask: bufstart = 0; bufvoid = 104857600 
2025-04-23 15:24:52,536 INFO mapred.MapTask: kvstart = 26214396; length = 6553600 
2025-04-23 15:24:52,538 INFO mapred.MapTask: Map output collector class = 
org.apache.hadoop.mapred.MapTask$MapOutputBuffer 
2025-04-23 15:24:52,545 INFO streaming.PipeMapRed: PipeMapRed exec 
[/home/hduser/./mapper.py] 
2025-04-23 15:24:52,547 INFO Configuration.deprecation: mapred.work.output.dir is 
deprecated. Instead, use mapreduce.task.output.dir 
2025-04-23 15:24:52,548 INFO Configuration.deprecation: mapred.local.dir is 
deprecated. Instead, use mapreduce.cluster.local.dir 
2025-04-23 15:24:52,549 INFO Configuration.deprecation: map.input.file is 
deprecated. Instead, use mapreduce.map.input.file 
2025-04-23 15:24:52,549 INFO Configuration.deprecation: map.input.length is 
deprecated. Instead, use mapreduce.map.input.length 
2025-04-23 15:24:52,549 INFO Configuration.deprecation: mapred.job.id is 
deprecated. Instead, use mapreduce.job.id 
2025-04-23 15:24:52,550 INFO Configuration.deprecation: mapred.task.partition is 
deprecated. Instead, use mapreduce.task.partition 
2025-04-23 15:24:52,551 INFO Configuration.deprecation: map.input.start is 
deprecated. Instead, use mapreduce.map.input.start 
2025-04-23 15:24:52,551 INFO Configuration.deprecation: mapred.task.is.map is 
deprecated. Instead, use mapreduce.task.ismap 
2025-04-23 15:24:52,551 INFO Configuration.deprecation: mapred.task.id is 
deprecated. Instead, use mapreduce.task.attempt.id 
2025-04-23 15:24:52,552 INFO Configuration.deprecation: mapred.tip.id is 
deprecated. Instead, use mapreduce.task.id 
2025-04-23 15:24:52,552 INFO Configuration.deprecation: mapred.skip.on is 
deprecated. Instead, use mapreduce.job.skiprecords 
2025-04-23 15:24:52,553 INFO Configuration.deprecation: user.name is deprecated. 
Instead, use mapreduce.job.user.name 
2025-04-23 15:24:52,631 INFO streaming.PipeMapRed: R/W/S=1/0/0 in:NA [rec/s] 
out:NA [rec/s] 
2025-04-23 15:24:52,631 INFO streaming.PipeMapRed: R/W/S=10/0/0 in:NA [rec/s] 
out:NA [rec/s] 
2025-04-23 15:24:52,632 INFO streaming.PipeMapRed: R/W/S=100/0/0 in:NA [rec/s] 
out:NA [rec/s] 
2025-04-23 15:24:52,633 INFO streaming.PipeMapRed: Records R/W=100/1 
2025-04-23 15:24:52,634 INFO streaming.PipeMapRed: MRErrorThread done 
2025-04-23 15:24:52,636 INFO streaming.PipeMapRed: mapRedFinished 
2025-04-23 15:24:52,638 INFO mapred.LocalJobRunner:  
2025-04-23 15:24:52,638 INFO mapred.MapTask: Starting flush of map output 
2025-04-23 15:24:52,638 INFO mapred.MapTask: Spilling map output 
2025-04-23 15:24:52,638 INFO mapred.MapTask: bufstart = 0; bufend = 1185; bufvoid 
= 104857600 
2025-04-23 15:24:52,638 INFO mapred.MapTask: kvstart = 26214396(104857584); kvend 
= 26214000(104856000); length = 397/6553600 
2025-04-23 15:24:52,648 INFO mapred.MapTask: Finished spill 0 
2025-04-23 15:24:52,659 INFO mapred.Task: 
Task:attempt_local1388892213_0001_m_000000_0 is done. And is in the process of 
committing 
2025-04-23 15:24:52,662 INFO mapred.LocalJobRunner: Records R/W=100/1 
2025-04-23 15:24:52,662 INFO mapred.Task: Task 
'attempt_local1388892213_0001_m_000000_0' done. 
2025-04-23 15:24:52,666 INFO mapred.Task: Final Counters for 
attempt_local1388892213_0001_m_000000_0: Counters: 23 
File System Counters 
FILE: Number of bytes read=1736 
FILE: Number of bytes written=649844 
FILE: Number of read operations=0 
FILE: Number of large read operations=0 
FILE: Number of write operations=0 
HDFS: Number of bytes read=1685 
HDFS: Number of bytes written=0 
HDFS: Number of read operations=5 
HDFS: Number of large read operations=0 
HDFS: Number of write operations=1 
HDFS: Number of bytes read erasure-coded=0 
Map-Reduce Framework 
Map input records=100 
Map output records=100 
Map output bytes=1185 
Map output materialized bytes=1391 
Input split bytes=97 
Combine input records=0 
Spilled Records=100 
Failed Shuffles=0 
Merged Map outputs=0 
GC time elapsed (ms)=6 
Total committed heap usage (bytes)=196083712 
File Input Format Counters  
Bytes Read=1685 
2025-04-23 15:24:52,667 INFO mapred.LocalJobRunner: Finishing task: 
attempt_local1388892213_0001_m_000000_0 
2025-04-23 15:24:52,667 INFO mapred.LocalJobRunner: map task executor complete. 
2025-04-23 15:24:52,669 INFO mapred.LocalJobRunner: Waiting for reduce tasks 
2025-04-23 15:24:52,669 INFO mapred.LocalJobRunner: Starting task: 
attempt_local1388892213_0001_r_000000_0 
2025-04-23 15:24:52,675 INFO output.FileOutputCommitter: File Output Committer 
Algorithm version is 2 
2025-04-23 15:24:52,675 INFO output.FileOutputCommitter: FileOutputCommitter skip 
cleanup _temporary folders under output directory:false, ignore cleanup failures: 
false 
2025-04-23 15:24:52,675 INFO mapred.Task:  Using ResourceCalculatorProcessTree : 
[ ] 
2025-04-23 15:24:52,677 INFO mapred.ReduceTask: Using ShuffleConsumerPlugin: 
org.apache.hadoop.mapreduce.task.reduce.Shuffle@77b2b11f 
2025-04-23 15:24:52,678 WARN impl.MetricsSystemImpl: JobTracker metrics system 
already initialized! 
2025-04-23 15:24:52,691 INFO reduce.MergeManagerImpl: MergerManager: 
memoryLimit=1437178240, maxSingleShuffleLimit=359294560, mergeThreshold=948537664, 
ioSortFactor=10, memToMemMergeOutputsThreshold=10 
2025-04-23 15:24:52,692 INFO reduce.EventFetcher: 
attempt_local1388892213_0001_r_000000_0 Thread started: EventFetcher for fetching 
Map Completion Events 
2025-04-23 15:24:52,714 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle 
output of map attempt_local1388892213_0001_m_000000_0 decomp: 1387 len: 1391 to 
MEMORY 
2025-04-23 15:24:52,715 INFO reduce.InMemoryMapOutput: Read 1387 bytes from map
output for attempt_local1388892213_0001_m_000000_0 
2025-04-23 15:24:52,716 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map
output of size: 1387, inMemoryMapOutputs.size() -> 1, commitMemory -> 0, 
usedMemory ->1387 
2025-04-23 15:24:52,717 INFO reduce.EventFetcher: EventFetcher is interrupted.. 
Returning 
2025-04-23 15:24:52,717 INFO mapred.LocalJobRunner: 1 / 1 copied. 
2025-04-23 15:24:52,717 INFO reduce.MergeManagerImpl: finalMerge called with 1 
in-memory map-outputs and 0 on-disk map-outputs 
2025-04-23 15:24:52,723 INFO mapred.Merger: Merging 1 sorted segments 
2025-04-23 15:24:52,723 INFO mapred.Merger: Down to the last merge-pass, with 1 
segments left of total size: 1373 bytes 
2025-04-23 15:24:52,726 INFO reduce.MergeManagerImpl: Merged 1 segments, 1387 
bytes to disk to satisfy reduce memory limit 
2025-04-23 15:24:52,726 INFO reduce.MergeManagerImpl: Merging 1 files, 1391 bytes 
from disk 
2025-04-23 15:24:52,726 INFO reduce.MergeManagerImpl: Merging 0 segments, 0 bytes 
from memory into reduce 
2025-04-23 15:24:52,726 INFO mapred.Merger: Merging 1 sorted segments 
2025-04-23 15:24:52,726 INFO mapred.Merger: Down to the last merge-pass, with 1 
segments left of total size: 1373 bytes 
2025-04-23 15:24:52,727 INFO mapred.LocalJobRunner: 1 / 1 copied. 
2025-04-23 15:24:52,729 INFO streaming.PipeMapRed: PipeMapRed exec 
[/home/hduser/./reducer.py] 
2025-04-23 15:24:52,731 INFO Configuration.deprecation: mapred.job.tracker is 
deprecated. Instead, use mapreduce.jobtracker.address 
2025-04-23 15:24:52,734 INFO Configuration.deprecation: mapred.map.tasks is 
deprecated. Instead, use mapreduce.job.maps 
2025-04-23 15:24:52,766 INFO streaming.PipeMapRed: R/W/S=1/0/0 in:NA [rec/s] 
out:NA [rec/s] 
2025-04-23 15:24:52,766 INFO streaming.PipeMapRed: R/W/S=10/0/0 in:NA [rec/s] 
out:NA [rec/s] 
2025-04-23 15:24:52,767 INFO streaming.PipeMapRed: R/W/S=100/0/0 in:NA [rec/s] 
out:NA [rec/s] 
2025-04-23 15:24:52,769 INFO streaming.PipeMapRed: Records R/W=100/1 
2025-04-23 15:24:52,771 INFO streaming.PipeMapRed: MRErrorThread done 
2025-04-23 15:24:52,771 INFO streaming.PipeMapRed: mapRedFinished 
2025-04-23 15:24:52,809 INFO mapred.Task: 
Task:attempt_local1388892213_0001_r_000000_0 is done. And is in the process of 
committing 
2025-04-23 15:24:52,812 INFO mapred.LocalJobRunner: 1 / 1 copied. 
2025-04-23 15:24:52,812 INFO mapred.Task: Task 
attempt_local1388892213_0001_r_000000_0 is allowed to commit now 
2025-04-23 15:24:52,826 INFO output.FileOutputCommitter: Saved output of task 
'attempt_local1388892213_0001_r_000000_0' to 
hdfs://localhost:54310/output/weather_output 
2025-04-23 15:24:52,827 INFO mapred.LocalJobRunner: Records R/W=100/1 > reduce 
2025-04-23 15:24:52,827 INFO mapred.Task: Task 
'attempt_local1388892213_0001_r_000000_0' done. 
2025-04-23 15:24:52,827 INFO mapred.Task: Final Counters for 
attempt_local1388892213_0001_r_000000_0: Counters: 30 
File System Counters 
FILE: Number of bytes read=4550 
FILE: Number of bytes written=651235 
FILE: Number of read operations=0 
FILE: Number of large read operations=0 
FILE: Number of write operations=0 
HDFS: Number of bytes read=1685 
HDFS: Number of bytes written=120 
HDFS: Number of read operations=10 
HDFS: Number of large read operations=0 
HDFS: Number of write operations=3 
HDFS: Number of bytes read erasure-coded=0 
Map-Reduce Framework 
Combine input records=0 
  Combine output records=0 
  Reduce input groups=100 
  Reduce shuffle bytes=1391 
  Reduce input records=100 
  Reduce output records=10 
  Spilled Records=100 
  Shuffled Maps =1 
  Failed Shuffles=0 
  Merged Map outputs=1 
  GC time elapsed (ms)=0 
  Total committed heap usage (bytes)=196083712 
 Shuffle Errors 
  BAD_ID=0 
  CONNECTION=0 
  IO_ERROR=0 
  WRONG_LENGTH=0 
  WRONG_MAP=0 
  WRONG_REDUCE=0 
 File Output Format Counters  
  Bytes Written=120 
2025-04-23 15:24:52,828 INFO mapred.LocalJobRunner: Finishing task: 
attempt_local1388892213_0001_r_000000_0 
2025-04-23 15:24:52,828 INFO mapred.LocalJobRunner: reduce task executor complete. 
2025-04-23 15:24:53,412 INFO mapreduce.Job: Job job_local1388892213_0001 running 
in uber mode : false 
2025-04-23 15:24:53,414 INFO mapreduce.Job:  map 100% reduce 100% 
2025-04-23 15:24:53,416 INFO mapreduce.Job: Job job_local1388892213_0001 
completed successfully 
2025-04-23 15:24:53,435 INFO mapreduce.Job: Counters: 36 
 File System Counters 
  FILE: Number of bytes read=6286 
  FILE: Number of bytes written=1301079 
  FILE: Number of read operations=0 
  FILE: Number of large read operations=0 
  FILE: Number of write operations=0 
  HDFS: Number of bytes read=3370 
  HDFS: Number of bytes written=120 
  HDFS: Number of read operations=15 
  HDFS: Number of large read operations=0 
  HDFS: Number of write operations=4 
  HDFS: Number of bytes read erasure-coded=0 
 Map-Reduce Framework 
  Map input records=100 
  Map output records=100 
  Map output bytes=1185 
  Map output materialized bytes=1391 
  Input split bytes=97 
  Combine input records=0 
  Combine output records=0 
  Reduce input groups=100 
  Reduce shuffle bytes=1391 
  Reduce input records=100 
  Reduce output records=10 
  Spilled Records=200 
  Shuffled Maps =1 
  Failed Shuffles=0 
  Merged Map outputs=1 
  GC time elapsed (ms)=6 
  Total committed heap usage (bytes)=392167424 
 Shuffle Errors 
  BAD_ID=0 
  CONNECTION=0 
  IO_ERROR=0 
  WRONG_LENGTH=0 
  WRONG_MAP=0 
  WRONG_REDUCE=0 
 File Input Format Counters  
  Bytes Read=1685 
 File Output Format Counters  
  Bytes Written=120 
2025-04-23 15:24:53,435 INFO streaming.StreamJob: Output directory: 
/output/weather_output 
7. View Output 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ hdfs dfs -ls /output/weather_output/ 
2025-04-23 15:25:17,254 WARN util.NativeCodeLoader: Unable to load native-hadoop 
library for your platform... using builtin-java classes where applicable 
Found 2 items -rw-r--r--   1 hduser supergroup          0 2025-04-23 15:24 
/output/weather_output/_SUCCESS -rw-r--r--   1 hduser supergroup        120 2025-04-23 15:24 
/output/weather_output/part-00000 
 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ hdfs dfs -cat 
/output/weather_output/part-00000 
2025-04-23 15:26:12,583 WARN util.NativeCodeLoader: Unable to load native-hadoop 
library for your platform... using builtin-java classes where applicable 
1950 -18 43 
1951 -17 44 
1952 -12 32 
1953 -20 41 
1954 -13 40 
1955 -16 45 
1956 -14 33 
1957 -19 38 
1958 -20 28 
1959 -19 40 
 
8. Stop HDFS 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ stop-dfs.sh  
Stopping namenodes on [localhost]  
Stopping datanodes  
Stopping secondary namenodes [Ubuntu]  
2025-04-23 15:26:40,461 WARN util.NativeCodeLoader: Unable to load native-hadoop 
library for your platform... using builtin-java classes where applicable  
 
hduser@pvg-HP-ProDesk-400-G4-SFF:~$ stop-yarn.sh  
Stopping nodemanagers  
Stopping resourcemanager